In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.06664323806762695
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 13.01161210545662
200000 24.21602365478761
300000 26.318188423172614
400000 32.26396912565237
500000 36.00944881606761
600000 36.2529217022556
700000 36.29729526980806
800000 35.221110329427255
900000 36.876055836694476
1000000 35.6008676385232
1100000 33.352711310794426
1200000 14.40975031229039
1300000 28.478511856437624
1400000 34.43457805680921
1500000 36.23823348645292
1600000 30.907332281325832
1700000 34.21218921137234
1800000 36.401120218736644
1900000 34.67729834100033
2000000 28.39836273045495
2100000 30.817914348685868
2200000 29.062241370114588
2300000 31.23446530543829
2400000 7.480586684758798
2500000 30.700416210931284
2600000 38.56927598992116
2700000 37.56906056764719
2800000 35.259720431078414
2900000 36.83608539264062
3000000 34.06666413

29500000 18.083479876052422
29600000 26.7186638735163
29700000 28.854741490439135
29800000 28.930341726919348
29900000 29.765130449665538
30000000 28.744091626799186
30100000 29.910772039265268
30200000 30.240839336222816
30300000 29.22813192403472
30400000 13.196259541607368
30500000 28.60099333509795
30600000 30.412195448220423
30700000 30.478806950015045
30800000 27.37403832427233
30900000 14.79016947947638
31000000 29.1742838281758
31100000 31.402107989192277
31200000 29.876080379794743
31300000 29.46612069753615
31400000 28.38139123158879
31500000 29.01771543470008
31600000 27.083176621513278
31700000 28.066116905562843
31800000 7.8669856390476625
31900000 28.190373419502407
32000000 26.87667877379179
32100000 27.215874203822853
32200000 29.28217444182608
32300000 28.387143452546333
32400000 29.640425119552482
32500000 29.6843735166359
32600000 29.830924231757322
32700000 19.31989156509907
32800000 23.189263184645764
32900000 29.01090302667687
33000000 31.306020106604812
33100000 

59200000 27.366267899198625
59300000 13.183529317854697
59400000 24.554853928817437
59500000 26.456583890930087
59600000 25.888156571868226
59700000 23.639050919078823
59800000 24.661230741360463
59900000 23.49930966023089
60000000 15.260682030317582
60100000 17.312325069979256
60200000 23.828252084655208
60300000 25.255665753243182
60400000 23.90147332799313
60500000 26.14924248301083
60600000 24.693339286991844
60700000 22.967787201084377
60800000 16.2506542554523
60900000 27.255682750221993
61000000 28.176958889660128
61100000 26.712141728087847
61200000 27.796970545764708
61300000 27.577273913933364
61400000 28.001987836296536
61500000 28.400048356187202
61600000 26.06371802047192
61700000 10.120447558652044
61800000 27.517550833790523
61900000 25.661916255927604
62000000 26.031016344686673
62100000 24.530865058628855
62200000 25.242039110774794
62300000 24.710062784998318
62400000 5.895322137407033
62500000 26.712278744940694
62600000 23.541229422833784
62700000 25.593409639865406

88900000 22.905895216933782
89000000 22.473340980281453
89100000 24.036962634610386
89200000 23.59309735354766
89300000 24.153647229007376
89400000 23.61803370549716
89500000 25.674311406423453
89600000 23.93842567638163
89700000 25.86489896203187
89800000 23.543447444501286
89900000 24.7514786079669
90000000 21.9411686326352
90100000 13.62792429849699
90200000 17.86815543311037
90300000 21.202641412937286
90400000 23.954381955730646
90500000 21.52420744782085
90600000 10.186242515866784
90700000 20.007360212944338
90800000 21.54304308622058
90900000 22.65506919206133
91000000 21.222932384007613
91100000 10.955710634809648
91200000 21.32035764805279
91300000 22.754272613266114
91400000 23.836584413059438
91500000 23.5937695018165
91600000 24.753927243592194
91700000 23.557300635203063
91800000 25.552394862119858
91900000 23.32912400111574
92000000 24.95042450224427
92100000 23.6964558714642
92200000 24.672156335589378
92300000 22.306242583586425
92400000 23.486319422643398
92500000 7.6

118000000 21.10087150913868
118100000 19.912249509322145
118200000 21.109686411393156
118300000 19.16664373285328
118400000 20.685419260177294
118500000 16.69409746373423
118600000 9.42708504904115
118700000 21.06370634928614
118800000 13.374801937879896
118900000 11.516516164008165
119000000 20.323872926963194
119100000 11.964258488801676
119200000 16.439193186112238
119300000 19.686469989828858
119400000 18.80628057818741
119500000 21.017145092042387
119600000 19.670032206773868
119700000 21.97596653217752
119800000 20.683594836529327
119900000 22.40712065681049
120000000 20.738981205128113
120100000 22.82807277419895
120200000 20.84739516477956
120300000 21.796409511146187
120400000 19.297846768766796
120500000 20.847600883728965
120600000 19.259011338546763
120700000 20.000989979134538
120800000 18.660369592575233
120900000 12.299702074132341
121000000 14.90553651760803
121100000 18.11184603191216
121200000 5.558881975632888
121300000 19.97139742219351
121400000 16.537107721948942


...

261400000 10.644311756089165
261500000 8.529347650240217
261600000 5.644518474256226
261700000 10.524615881944648
261800000 9.38098327185285
261900000 9.98155971124702
262000000 9.36273737801351
262100000 9.624554735247969
262200000 9.006316947145551
262300000 7.585677890008225
262400000 5.022042051342655
262500000 8.770196414336283
262600000 9.55543863922619
262700000 9.271079620064432
262800000 10.018861775674239
262900000 9.41851218092553
263000000 10.470087862300229
263100000 8.13290821322082
263200000 5.724289890940464
263300000 10.75745099913173
263400000 9.743842645186103
263500000 10.70951341455634
263600000 10.01667807682156
263700000 10.485974128841155
263800000 10.042633267242348
263900000 4.183359785746168
264000000 11.128356833502401
264100000 10.400872082580953
264200000 9.506625432732239
264300000 9.905476588643895
264400000 9.653011709167766
264500000 9.251532519063314
264600000 9.481790727486553
264700000 3.641318694252491
264800000 9.02090234196781
264900000 9.2394688

290800000 8.088033290884573
290900000 7.646647284675378
291000000 8.118577444388437
291100000 7.52655423186636
291200000 8.15990824850558
291300000 7.762815867321097
291400000 8.275083703045102
291500000 7.995464180703618
291600000 2.775742972153382
291700000 8.61929245292666
291800000 7.589698498920743
291900000 4.710402002618613
292000000 6.961929819538071
292100000 8.204733727821587
292200000 7.433117339165472
292300000 8.099284356535067
292400000 7.521722667678766
292500000 8.009166363053467
292600000 7.6866159816553115
292700000 8.010503962563279
292800000 7.338442107270712
292900000 4.803161029568279
293000000 5.165558356909437
293100000 7.488842995746647
293200000 8.217675552528647
293300000 7.573540019684119
293400000 8.195369909017701
293500000 7.7949280152727605
293600000 8.307233580236726
293700000 7.474987183232354
293800000 8.22716825773666
293900000 7.217521987990703
294000000 3.9795472993784458
294100000 7.544256213641662
294200000 6.426805718196178
294300000 4.908725022

319600000 2.4173274447968063
319700000 2.621133878775966
319800000 2.3854928288968296
319900000 2.494141526504622
320000000 2.633255199100966
320100000 2.805845536321254
320200000 2.6171194078727287
320300000 2.2855709560145043
320400000 2.2879910201781657
320500000 2.7137224787575662
320600000 2.3744127660908316
320700000 2.0103518022340543
320800000 2.31314431112441
320900000 2.0853812819907787
321000000 2.0185188835620895
321100000 1.2447501817962605
321200000 1.232180435513262
321300000 1.2396129707688484
321400000 1.4554442075745762
321500000 2.377038737322312
321600000 2.9455747891301023
321700000 2.2056591863267085
321800000 1.9204608683374664
321900000 2.3339194985311638
322000000 2.3092131864283463
322100000 2.587469030564739
322200000 2.252196240855057
322300000 2.5832510377246853
322400000 2.3666749957990096
322500000 2.7570156561139
322600000 2.5087986801421085
322700000 2.302821828658012
322800000 2.0827533632417956
322900000 2.139717173319964
323000000 2.3754474024480956


348200000 1.4959767646539388
348300000 1.5604347196906188
348400000 1.3727535570580722
348500000 1.5080036284259664
348600000 1.307018142752758
348700000 1.3732924927235304
348800000 1.1173861020494409
348900000 1.3656848187679862
349000000 1.4832981024758933
349100000 1.3786441856937255
349200000 0.9530893841289811
349300000 0.649917281874615
349400000 0.6947372572437108
349500000 0.9024645635902379
349600000 0.8998617437976419
349700000 1.0552031312059322
349800000 1.4391027045800124
349900000 1.1767063926453531
350000000 1.4430121288413325
350100000 1.3399152946397113
350200000 1.2404631054154218
350300000 1.2691821995537784
350400000 1.4463992232959755
350500000 1.4054883007742687
350600000 1.4310416901532215
350700000 1.3250838450801485
350800000 1.3406225658971886
350900000 1.4036643758418714
351000000 1.194936946892682
351100000 1.216980423454076
351200000 1.2451605663082115
351300000 1.3538331840268016
351400000 1.2387293763485412
351500000 1.010176177067797
351600000 0.8242335

376000000 0.0489868160360674
376100000 0.033613850724817644
376200000 0.03386074194848868
376300000 0.05089083612554669
376400000 0.05186688697574801
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.175636082547466
376600000 0.175401144949158
376700000 0.3142000936342233
376800000 0.18585806010780598
376900000 0.16661783127267757
377000000 0.17781579570122824
377100000 0.1854328556085547
377200000 0.22168137779693603
377300000 0.2179759831531816
377400000 0.17313038297629157
377500000 0.12812416690793038
377600000 0.12281918207724225
377700000 0.12885716846700113
377800000 0.1985205517004331
377900000 0.1383940243135598
378000000 0.21782492005652984
378100000 0.1853873030290604
378200000 0.20131250192552008
378300000 0.12163848051408728
378400000 0.13942202716618773
378500000 0.1505618360057725
378600000 0.1734793416209393
378700000 0.2080497110623499
378800000 0.13835622435467113
378900000 0.03257386871165236
379000000 0.02685664481602907
379100000 0.0